In [2]:
import pandas as pd # package for high-performance, easy-to-use data structures and data analysis


In [10]:
## Load the DATA
app_train = pd.read_csv('/Users/mauroaz/Downloads/home-credit-default-risk_02/application_train.csv')
pos_cash_balance = pd.read_csv('/Users/mauroaz/Downloads/home-credit-default-risk_02/POS_CASH_balance.csv')
bureau_balance = pd.read_csv('/Users/mauroaz/Downloads/home-credit-default-risk_02/bureau_balance.csv')
previous_app = pd.read_csv('/Users/mauroaz/Downloads/home-credit-default-risk_02/previous_application.csv')
installments_payments = pd.read_csv('/Users/mauroaz/Downloads/home-credit-default-risk_02/installments_payments.csv')
credit_card_balance = pd.read_csv('/Users/mauroaz/Downloads/home-credit-default-risk_02/credit_card_balance.csv')
bureau = pd.read_csv('/Users/mauroaz/Downloads/home-credit-default-risk_02/bureau.csv')
app_test = pd.read_csv('/Users/mauroaz/Downloads/home-credit-default-risk_02/application_test.csv')



In [4]:
import numpy as np # linear algebra

import warnings
warnings.filterwarnings('ignore')

In [11]:
# Groupby the client id (SK_ID_CURR), count the number of previous loans, and rename the column
previous_loan_counts = bureau.groupby('SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count().rename(columns = {'SK_ID_BUREAU': 'previous_loan_counts'})
previous_loan_counts.head()

,SK_ID_CURR,previous_loan_counts
0,100001,7
1,100002,8
2,100003,4
3,100004,2
4,100005,3


In [12]:
# Function to count and normalize values of categorical variables ¶

def normalize_categorical(df, group_var, col_name):
    
    """Computes counts and normalized counts for each observation
    of `group_var` for each unique category in every categorical variable
    
    Parameters 
    ----------
    df - DataFrame for which we will calculate count
    
    group_var  = string
        The variable by which to group the dataframe. For each unique
        value of this variable, the final dataframe will have one row
        
    col_name = string
            Variable added to the front of column names to keep track of columns
            
            """
    # select the categorical columns
    categorical = pd.get_dummies(df.select_dtypes('object'))
    
    # Make sure to put the identifying id on the column
    categorical[group_var] = df[group_var]
    
    # Groupby the group var and calculate the sum and mean
    categorical = categorical.groupby(group_var).agg(['sum', 'mean'])                                              
    
    column_names = []
    
    # Iterate through the columns in level 0
    for var in categorical.columns.levels[0]:
        # Iterate through the stats in level 1
        for stat in ['count', 'count_norm']:
            # Make a new column name
            column_names.append('%s_%s_%s' % (col_name, var, stat))
    
    categorical.columns = column_names
    
    return categorical

In [13]:
bureau_counts = normalize_categorical(bureau, group_var = 'SK_ID_CURR', col_name = 'bureau')
bureau_counts.head()

,bureau_CREDIT_ACTIVE_Active_count,bureau_CREDIT_ACTIVE_Active_count_norm,bureau_CREDIT_ACTIVE_Bad debt_count,bureau_CREDIT_ACTIVE_Bad debt_count_norm,bureau_CREDIT_ACTIVE_Closed_count,bureau_CREDIT_ACTIVE_Closed_count_norm,bureau_CREDIT_ACTIVE_Sold_count,bureau_CREDIT_ACTIVE_Sold_count_norm,bureau_CREDIT_CURRENCY_currency 1_count,bureau_CREDIT_CURRENCY_currency 1_count_norm,...,bureau_CREDIT_TYPE_Microloan_count,bureau_CREDIT_TYPE_Microloan_count_norm,bureau_CREDIT_TYPE_Mobile operator loan_count,bureau_CREDIT_TYPE_Mobile operator loan_count_norm,bureau_CREDIT_TYPE_Mortgage_count,bureau_CREDIT_TYPE_Mortgage_count_norm,bureau_CREDIT_TYPE_Real estate loan_count,bureau_CREDIT_TYPE_Real estate loan_count_norm,bureau_CREDIT_TYPE_Unknown type of loan_count,bureau_CREDIT_TYPE_Unknown type of loan_count_norm
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,3,0.428571,0,0.0,4,0.571429,0,0.0,7,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100002,2,0.250000,0,0.0,6,0.750000,0,0.0,8,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100003,1,0.250000,0,0.0,3,0.750000,0,0.0,4,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100004,0,0.000000,0,0.0,2,1.000000,0,0.0,2,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100005,2,0.666667,0,0.0,1,0.333333,0,0.0,3,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0


In [14]:
 ## Grouping data  so  that we can merge all the files in 1 dataset

data_bureau_agg=bureau.groupby(by='SK_ID_CURR').mean()
data_credit_card_balance_agg=credit_card_balance.groupby(by='SK_ID_CURR').mean()
data_previous_application_agg=previous_app.groupby(by='SK_ID_CURR').mean()
data_installments_payments_agg=installments_payments.groupby(by='SK_ID_CURR').mean()
data_POS_CASH_balance_agg=pos_cash_balance.groupby(by='SK_ID_CURR').mean()

data_bureau_agg.head()

,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
SK_ID_CURR,,,,,,,,,,,,,
100001,5896633.000,-735.000000,0.0,82.428571,-825.500000,NaN,0.0,207623.571429,85240.928571,0.00000,0.0,-93.142857,3545.357143
100002,6153272.125,-874.000000,0.0,-349.000000,-697.500000,1681.029,0.0,108131.945625,49156.200000,7997.14125,0.0,-499.875000,0.000000
100003,5885878.500,-1400.750000,0.0,-544.500000,-1097.333333,0.000,0.0,254350.125000,0.000000,202500.00000,0.0,-816.000000,NaN
100004,6829133.500,-867.000000,0.0,-488.500000,-532.500000,0.000,0.0,94518.900000,0.000000,0.00000,0.0,-532.000000,NaN
100005,6735201.000,-190.666667,0.0,439.333333,-123.000000,0.000,0.0,219042.000000,189469.500000,0.00000,0.0,-54.333333,1420.500000


In [15]:
#MERGE all the data
def merge(df):
    df = df.join(data_bureau_agg, how='left', on='SK_ID_CURR', lsuffix='1', rsuffix='2') 
    df = df.join(bureau_counts, on = 'SK_ID_CURR', how = 'left')
    df = df.merge(previous_loan_counts, on = 'SK_ID_CURR', how = 'left')
    df = df.join(data_credit_card_balance_agg, how='left', on='SK_ID_CURR', lsuffix='1', rsuffix='2')    
    df = df.join(data_previous_application_agg, how='left', on='SK_ID_CURR', lsuffix='1', rsuffix='2')   
    df = df.join(data_installments_payments_agg, how='left', on='SK_ID_CURR', lsuffix='1', rsuffix='2') 
    
    return df

train = merge(app_train)
test = merge(app_test)
display(train.head())

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT1,AMT_ANNUITY1,...,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,SK_ID_PREV,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,-25.000000,-17.000000,0.000000,1.038818e+06,1.052632,10.000000,-295.000000,-315.421053,11559.247105,11559.247105
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,-1054.333333,-1047.333333,0.666667,2.290070e+06,1.040000,5.080000,-1378.160000,-1385.320000,64754.586000,64754.586000
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,-724.000000,-714.000000,0.000000,1.564014e+06,1.333333,2.000000,-754.000000,-761.666667,7096.155000,7096.155000
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,182477.500000,182481.750000,0.000000,2.217428e+06,1.125000,4.437500,-252.250000,-271.625000,62947.088438,62947.088438
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,72136.200000,72143.800000,0.600000,2.048985e+06,1.166667,7.045455,-1028.606061,-1032.242424,12666.444545,12214.060227


In [16]:
print(train.shape)
print(test.shape)

(307511, 230)
(48744, 229)


In [17]:
#combining the data
ntrain = train.shape[0]
ntest = test.shape[0]

y_train = train.TARGET.values

#train_df = train_df.drop

all_data = pd.concat([train, test]).reset_index(drop=True)
all_data.drop(['TARGET'], axis=1, inplace=True)

In [ ]:
# Feature Engineering ...Adding Extra stuff


In [18]:
# Now we will convert days employed and days registration and days id publish to a positive no. 
def correct_birth(df):
    
    df['DAYS_BIRTH'] = round((df['DAYS_BIRTH'] * (-1))/365)
    return df

def convert_abs(df):
    df['DAYS_EMPLOYED'] = abs(df['DAYS_EMPLOYED'])
    df['DAYS_REGISTRATION'] = abs(df['DAYS_REGISTRATION'])
    df['DAYS_ID_PUBLISH'] = abs(df['DAYS_ID_PUBLISH'])
    df['DAYS_LAST_PHONE_CHANGE'] = abs(df['DAYS_LAST_PHONE_CHANGE'])
    return df

# Now we will fill misisng values in OWN_CAR_AGE. 
#Most probably there will be missing values if the person does not own a car. So we will fill with 0

def missing(df):
    
    features = ['previous_loan_counts','NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAPARTMENTS_AVG','NONLIVINGAREA_MEDI','OWN_CAR_AGE']
    
    for f in features:
        df[f] = df[f].fillna(0 )
    return df

def transform_app(df):
    df = correct_birth(df)
    df = convert_abs(df)
    df = missing(df)
    return df

   

all_data = transform_app(all_data)

In [19]:
# counting no of phones given by the company and delete the irrelevant features
all_data['NO_OF_CLIENT_PHONES'] = all_data['FLAG_MOBIL'] + all_data['FLAG_EMP_PHONE'] + all_data['FLAG_WORK_PHONE']
all_data.head()

,AMT_ANNUITY,AMT_ANNUITY1,AMT_ANNUITY2,AMT_APPLICATION,AMT_BALANCE,AMT_CREDIT1,AMT_CREDIT2,AMT_CREDIT_LIMIT_ACTUAL,AMT_CREDIT_MAX_OVERDUE,AMT_CREDIT_SUM,...,bureau_CREDIT_TYPE_Mobile operator loan_count,bureau_CREDIT_TYPE_Mobile operator loan_count_norm,bureau_CREDIT_TYPE_Mortgage_count,bureau_CREDIT_TYPE_Mortgage_count_norm,bureau_CREDIT_TYPE_Real estate loan_count,bureau_CREDIT_TYPE_Real estate loan_count_norm,bureau_CREDIT_TYPE_Unknown type of loan_count,bureau_CREDIT_TYPE_Unknown type of loan_count_norm,previous_loan_counts,NO_OF_CLIENT_PHONES
0,9251.775,24700.5,0.0,179055.00,NaN,406597.5,179055.00,NaN,1681.029,108131.945625,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2
1,56553.990,35698.5,NaN,435436.50,NaN,1293502.5,484191.00,NaN,0.000,254350.125000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2
2,5357.250,6750.0,NaN,24282.00,NaN,135000.0,20106.00,NaN,0.000,94518.900000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3
3,23651.175,29686.5,NaN,272203.26,0.0,312682.5,291695.50,270000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2
4,12278.805,21865.5,NaN,150530.25,NaN,513000.0,166638.75,NaN,0.000,146250.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2


In [20]:
# add a feature to determine if client's permanent city does not match with contact/work city
all_data['FLAG_CLIENT_OUTSIDE_CITY'] = np.where((all_data['REG_CITY_NOT_WORK_CITY']==1) & (all_data['REG_CITY_NOT_LIVE_CITY']==1),1,0)
all_data.head()

,AMT_ANNUITY,AMT_ANNUITY1,AMT_ANNUITY2,AMT_APPLICATION,AMT_BALANCE,AMT_CREDIT1,AMT_CREDIT2,AMT_CREDIT_LIMIT_ACTUAL,AMT_CREDIT_MAX_OVERDUE,AMT_CREDIT_SUM,...,bureau_CREDIT_TYPE_Mobile operator loan_count_norm,bureau_CREDIT_TYPE_Mortgage_count,bureau_CREDIT_TYPE_Mortgage_count_norm,bureau_CREDIT_TYPE_Real estate loan_count,bureau_CREDIT_TYPE_Real estate loan_count_norm,bureau_CREDIT_TYPE_Unknown type of loan_count,bureau_CREDIT_TYPE_Unknown type of loan_count_norm,previous_loan_counts,NO_OF_CLIENT_PHONES,FLAG_CLIENT_OUTSIDE_CITY
0,9251.775,24700.5,0.0,179055.00,NaN,406597.5,179055.00,NaN,1681.029,108131.945625,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2,0
1,56553.990,35698.5,NaN,435436.50,NaN,1293502.5,484191.00,NaN,0.000,254350.125000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2,0
2,5357.250,6750.0,NaN,24282.00,NaN,135000.0,20106.00,NaN,0.000,94518.900000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3,0
3,23651.175,29686.5,NaN,272203.26,0.0,312682.5,291695.50,270000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2,0
4,12278.805,21865.5,NaN,150530.25,NaN,513000.0,166638.75,NaN,0.000,146250.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0


In [21]:
 # add a feature to determine if client's permanent city does not match with contact/work region
all_data['FLAG_CLIENT_OUTSIDE_REGION'] = np.where((all_data['REG_REGION_NOT_LIVE_REGION']==1) & (all_data['REG_REGION_NOT_WORK_REGION']==1),1,0)
all_data.head()

,AMT_ANNUITY,AMT_ANNUITY1,AMT_ANNUITY2,AMT_APPLICATION,AMT_BALANCE,AMT_CREDIT1,AMT_CREDIT2,AMT_CREDIT_LIMIT_ACTUAL,AMT_CREDIT_MAX_OVERDUE,AMT_CREDIT_SUM,...,bureau_CREDIT_TYPE_Mortgage_count,bureau_CREDIT_TYPE_Mortgage_count_norm,bureau_CREDIT_TYPE_Real estate loan_count,bureau_CREDIT_TYPE_Real estate loan_count_norm,bureau_CREDIT_TYPE_Unknown type of loan_count,bureau_CREDIT_TYPE_Unknown type of loan_count_norm,previous_loan_counts,NO_OF_CLIENT_PHONES,FLAG_CLIENT_OUTSIDE_CITY,FLAG_CLIENT_OUTSIDE_REGION
0,9251.775,24700.5,0.0,179055.00,NaN,406597.5,179055.00,NaN,1681.029,108131.945625,...,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2,0,0
1,56553.990,35698.5,NaN,435436.50,NaN,1293502.5,484191.00,NaN,0.000,254350.125000,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2,0,0
2,5357.250,6750.0,NaN,24282.00,NaN,135000.0,20106.00,NaN,0.000,94518.900000,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3,0,0
3,23651.175,29686.5,NaN,272203.26,0.0,312682.5,291695.50,270000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2,0,0
4,12278.805,21865.5,NaN,150530.25,NaN,513000.0,166638.75,NaN,0.000,146250.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0,0


# Deleting Useless Features

In [22]:
# deleting useless features
def delete(df):
   # useless=['FLAG_MOBIL', 'FLAG_EMP_PHONE' ,'FLAG_WORK_PHONE','REG_CITY_NOT_WORK_CITY','REG_CITY_NOT_LIVE_CITY','REG_REGION_NOT_LIVE_REGION','REG_REGION_NOT_WORK_REGION']
    #for feature in useless:
     return df.drop(['FLAG_MOBIL', 'FLAG_EMP_PHONE' ,'FLAG_WORK_PHONE','REG_CITY_NOT_WORK_CITY','REG_CITY_NOT_LIVE_CITY','REG_REGION_NOT_LIVE_REGION','REG_REGION_NOT_WORK_REGION'], axis=1)
def transform(df):
   # df = convert_abs(df)
    df = delete(df)
   
    return df

all_data = transform(all_data)
all_data.head()

,AMT_ANNUITY,AMT_ANNUITY1,AMT_ANNUITY2,AMT_APPLICATION,AMT_BALANCE,AMT_CREDIT1,AMT_CREDIT2,AMT_CREDIT_LIMIT_ACTUAL,AMT_CREDIT_MAX_OVERDUE,AMT_CREDIT_SUM,...,bureau_CREDIT_TYPE_Mortgage_count,bureau_CREDIT_TYPE_Mortgage_count_norm,bureau_CREDIT_TYPE_Real estate loan_count,bureau_CREDIT_TYPE_Real estate loan_count_norm,bureau_CREDIT_TYPE_Unknown type of loan_count,bureau_CREDIT_TYPE_Unknown type of loan_count_norm,previous_loan_counts,NO_OF_CLIENT_PHONES,FLAG_CLIENT_OUTSIDE_CITY,FLAG_CLIENT_OUTSIDE_REGION
0,9251.775,24700.5,0.0,179055.00,NaN,406597.5,179055.00,NaN,1681.029,108131.945625,...,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2,0,0
1,56553.990,35698.5,NaN,435436.50,NaN,1293502.5,484191.00,NaN,0.000,254350.125000,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2,0,0
2,5357.250,6750.0,NaN,24282.00,NaN,135000.0,20106.00,NaN,0.000,94518.900000,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3,0,0
3,23651.175,29686.5,NaN,272203.26,0.0,312682.5,291695.50,270000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2,0,0
4,12278.805,21865.5,NaN,150530.25,NaN,513000.0,166638.75,NaN,0.000,146250.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0,0


In [23]:
# delete Ids

def delete_id(df):
    return df.drop(['SK_ID_CURR', 'SK_ID_PREV','SK_ID_BUREAU'], axis = 1)

all_data = delete_id(all_data)

In [24]:
all_data.head()

,AMT_ANNUITY,AMT_ANNUITY1,AMT_ANNUITY2,AMT_APPLICATION,AMT_BALANCE,AMT_CREDIT1,AMT_CREDIT2,AMT_CREDIT_LIMIT_ACTUAL,AMT_CREDIT_MAX_OVERDUE,AMT_CREDIT_SUM,...,bureau_CREDIT_TYPE_Mortgage_count,bureau_CREDIT_TYPE_Mortgage_count_norm,bureau_CREDIT_TYPE_Real estate loan_count,bureau_CREDIT_TYPE_Real estate loan_count_norm,bureau_CREDIT_TYPE_Unknown type of loan_count,bureau_CREDIT_TYPE_Unknown type of loan_count_norm,previous_loan_counts,NO_OF_CLIENT_PHONES,FLAG_CLIENT_OUTSIDE_CITY,FLAG_CLIENT_OUTSIDE_REGION
0,9251.775,24700.5,0.0,179055.00,NaN,406597.5,179055.00,NaN,1681.029,108131.945625,...,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2,0,0
1,56553.990,35698.5,NaN,435436.50,NaN,1293502.5,484191.00,NaN,0.000,254350.125000,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2,0,0
2,5357.250,6750.0,NaN,24282.00,NaN,135000.0,20106.00,NaN,0.000,94518.900000,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3,0,0
3,23651.175,29686.5,NaN,272203.26,0.0,312682.5,291695.50,270000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2,0,0
4,12278.805,21865.5,NaN,150530.25,NaN,513000.0,166638.75,NaN,0.000,146250.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0,0


# Now....

In [25]:
print(all_data.columns)

Index(['AMT_ANNUITY', 'AMT_ANNUITY1', 'AMT_ANNUITY2', 'AMT_APPLICATION',
       'AMT_BALANCE', 'AMT_CREDIT1', 'AMT_CREDIT2', 'AMT_CREDIT_LIMIT_ACTUAL',
       'AMT_CREDIT_MAX_OVERDUE', 'AMT_CREDIT_SUM',
       ...
       'bureau_CREDIT_TYPE_Mortgage_count',
       'bureau_CREDIT_TYPE_Mortgage_count_norm',
       'bureau_CREDIT_TYPE_Real estate loan_count',
       'bureau_CREDIT_TYPE_Real estate loan_count_norm',
       'bureau_CREDIT_TYPE_Unknown type of loan_count',
       'bureau_CREDIT_TYPE_Unknown type of loan_count_norm',
       'previous_loan_counts', 'NO_OF_CLIENT_PHONES',
       'FLAG_CLIENT_OUTSIDE_CITY', 'FLAG_CLIENT_OUTSIDE_REGION'],
      dtype='object', length=222)


In [26]:
# handling missing values

def miss_numerical(df):
    
    features = ['previous_loan_counts','NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAPARTMENTS_AVG','NONLIVINGAREA_MEDI','OWN_CAR_AGE']
    numerical_features = all_data.select_dtypes(exclude = ["object"] ).columns
    #print(numerical_features)
    for f in numerical_features:
        #print(f)
        if f not in features:
            df[f] = df[f].fillna(df[f].median())
      
    return df

def miss_categorical(df):
    
    categorical_features = all_data.select_dtypes(include = ["object"]).columns
    
    for f in categorical_features:
        df[f] = df[f].fillna(df[f].mode()[0])
        
    return df

def transform_feature(df):
    df = miss_numerical(df)
    df = miss_categorical(df)
    #df = fill_cabin(df)
    return df

all_data = transform_feature(all_data)


all_data.head()

,AMT_ANNUITY,AMT_ANNUITY1,AMT_ANNUITY2,AMT_APPLICATION,AMT_BALANCE,AMT_CREDIT1,AMT_CREDIT2,AMT_CREDIT_LIMIT_ACTUAL,AMT_CREDIT_MAX_OVERDUE,AMT_CREDIT_SUM,...,bureau_CREDIT_TYPE_Mortgage_count,bureau_CREDIT_TYPE_Mortgage_count_norm,bureau_CREDIT_TYPE_Real estate loan_count,bureau_CREDIT_TYPE_Real estate loan_count_norm,bureau_CREDIT_TYPE_Unknown type of loan_count,bureau_CREDIT_TYPE_Unknown type of loan_count_norm,previous_loan_counts,NO_OF_CLIENT_PHONES,FLAG_CLIENT_OUTSIDE_CITY,FLAG_CLIENT_OUTSIDE_REGION
0,9251.775,24700.5,0.0,179055.00,24997.602995,406597.5,179055.00,149000.0,1681.029,108131.945625,...,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2,0,0
1,56553.990,35698.5,6516.0,435436.50,24997.602995,1293502.5,484191.00,149000.0,0.000,254350.125000,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2,0,0
2,5357.250,6750.0,6516.0,24282.00,24997.602995,135000.0,20106.00,149000.0,0.000,94518.900000,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3,0,0
3,23651.175,29686.5,6516.0,272203.26,0.000000,312682.5,291695.50,270000.0,0.000,197297.250000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0,0
4,12278.805,21865.5,6516.0,150530.25,24997.602995,513000.0,166638.75,149000.0,0.000,146250.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0,0


In [27]:
# Scaling the data 

from sklearn.preprocessing import MinMaxScaler

def encoder(df):
    scaler = MinMaxScaler()
    numerical = all_data.select_dtypes(exclude = ["object"]).columns
    features_transform = pd.DataFrame(data= df)
    features_transform[numerical] = scaler.fit_transform(df[numerical])
    display(features_transform.head(n = 5))
    return df

all_data = encoder(all_data)

#display(all_data.head())

,AMT_ANNUITY,AMT_ANNUITY1,AMT_ANNUITY2,AMT_APPLICATION,AMT_BALANCE,AMT_CREDIT1,AMT_CREDIT2,AMT_CREDIT_LIMIT_ACTUAL,AMT_CREDIT_MAX_OVERDUE,AMT_CREDIT_SUM,...,bureau_CREDIT_TYPE_Mortgage_count,bureau_CREDIT_TYPE_Mortgage_count_norm,bureau_CREDIT_TYPE_Real estate loan_count,bureau_CREDIT_TYPE_Real estate loan_count_norm,bureau_CREDIT_TYPE_Unknown type of loan_count,bureau_CREDIT_TYPE_Unknown type of loan_count_norm,previous_loan_counts,NO_OF_CLIENT_PHONES,FLAG_CLIENT_OUTSIDE_CITY,FLAG_CLIENT_OUTSIDE_REGION
0,0.030796,0.090032,0.000000,0.044211,0.029978,0.090287,0.044211,0.11037,0.000014,0.000546,...,0.0,0.0,0.0,0.0,0.0,0.0,0.068966,0.666667,0.0,0.0
1,0.188246,0.132924,0.000239,0.107515,0.029978,0.311736,0.119553,0.11037,0.000000,0.001284,...,0.0,0.0,0.0,0.0,0.0,0.0,0.034483,0.666667,0.0,0.0
2,0.017832,0.020025,0.000239,0.005996,0.029978,0.022472,0.004964,0.11037,0.000000,0.000477,...,0.0,0.0,0.0,0.0,0.0,0.0,0.017241,1.000000,0.0,0.0
3,0.078726,0.109477,0.000239,0.067211,0.003145,0.066837,0.072024,0.20000,0.000000,0.000996,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.666667,0.0,0.0
4,0.040871,0.078975,0.000239,0.037168,0.029978,0.116854,0.041145,0.11037,0.000000,0.000738,...,0.0,0.0,0.0,0.0,0.0,0.0,0.008621,0.666667,0.0,0.0


# Converting Categoricals

In [28]:
# Converting into categorical features

# Create a label encoder object
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le_count = 0


# Iterate through the columns
for col in all_data:
    if all_data[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(all_data[col].unique())) <= 2:
            # Train on the training data
            le.fit(all_data[col])
            # Transform both training and testing data
            all_data[col] = le.transform(all_data[col])
            #test[col] = le.transform(test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
           
print('%d columns were label encoded.' % le_count)

4 columns were label encoded.


In [29]:
# dummy variables
all_data = pd.get_dummies(all_data)

display(all_data.shape)

(356255, 342)

# Modeling

In [31]:
### Splitting features
train = all_data[:ntrain]
test = all_data[ntrain:]

print("Training shape", train.shape)
print("Testing shape", test.shape)


Training shape (307511, 342)
Testing shape (48744, 342)


In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(train, y_train, test_size = 0.3, random_state = 200)
print("X Training shape", X_train.shape)
print("X Testing shape", X_test.shape)
print("Y Training shape", Y_train.shape)
print("Y Testing shape", Y_test.shape)

X Training shape (215257, 342)
X Testing shape (92254, 342)
Y Training shape (215257,)
Y Testing shape (92254,)


In [34]:
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc, log_loss
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
#from sklearn.grid_search import GridSearchCV
## https://stackoverflow.com/questions/54455632/importerror-no-module-named-grid-search-learning-curve/54461508
from sklearn.model_selection import learning_curve, GridSearchCV

logreg = LogisticRegression(random_state=0, class_weight='balanced', C=100)
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict_proba(X_test)[:,1]

#Y_pred_proba = logreg.predict_proba(X_test)

print('Train/Test split results:')
#print(logreg.__class__.__name__+" accuracy is %2.3f" % accuracy_score(Y_test, Y_pred))
print("ROC",  roc_auc_score(Y_test, Y_pred))
#print(logreg.__class__.__name__+" auc is %2.3f" % auc(fpr, tpr))

Train/Test split results:
ROC 0.7621211226134446


In [35]:
#### Try to improve model with EXTRA features.... ???

# How to time excution time ?

# Now lets run the Statmodel